In [1]:
using BayesianTools.ProductDistributions
p = ProductDistribution(Normal(0,1), Beta(1.,1.))
n = length(p) ## 2 -> Number of distributions in the product

┌ Info: Precompiling BayesianTools [196d93e9-2b0a-59bc-85dd-07b4a30d24dc]
└ @ Base loading.jl:1186


2

In [7]:
using Random
rand!(p, zeros(Float64,(2,100)))

2×100 Array{Float64,2}:
 -0.17021   -0.331844  -0.724829  -0.304572  …  -0.573888  0.264385  0.249806
  0.958178   0.933858   0.148761   0.514265      0.528566  0.504538  0.662535

In [10]:
using BayesianTools.Links
function mcmc_wrong(iters)
   chain = zeros(Float64, iters)
   gamma = Gamma(2, 1)
   d = Improper(0, +Inf)
   lx  = 1.0
   for i in 1:iters
      xs = link(d, lx) + randn()
      lxs = invlink(d, xs)
      a = logpdf(gamma, lxs)-logpdf(gamma, lx)       
      (rand() < exp(a)) && (lx = lxs)
      chain[i] = lx
   end
   return chain
end

mcmc_wrong (generic function with 1 method)

In [11]:
function mcmc_right(iters)
   chain = zeros(Float64,iters)
   gamma = Gamma(2, 1)
   d = Improper(0, +Inf)
   lx  = 1.0
   for i in 1:iters
      xs = link(d, lx) + randn()
      lxs = invlink(d, xs)
      a = logpdf(gamma, lxs)-logpdf(gamma, lx)
      ## Log absolute jacobian adjustment
      a = a - logjacobian(d, lxs) + logjacobian(d, lx)
      (rand() < exp(a)) && (lx = lxs)
      chain[i] = lx
   end
   return chain
end

mcmc_right (generic function with 1 method)

In [12]:
Plots.histogram([mc0, mc1], normalize=true, bins = 100, fill=:slategray, layout = (1,2), lab = "draws")



┌ Info: Recompiling stale cache file /Users/navaro/.julia/compiled/v1.1/Plots/ld3vC.ji for Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1184


LoadError: syntax: extra token ")" after end of expression